## Dynamic Programming

## - Fibonacci

In [1]:
# O(2^n) time
def fib(n): 
    if n == 1 or n == 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Memoization, O(n) time
def fib_memo(n):
    memo = [None] * (n + 1)
    
    def fib_memo_util(n, memo):
        if memo[n]:
            return memo[n]
        else:
            if n == 1 or n == 2:
                result = 1
            else:
                result = fib_memo_util(n-1, memo) + fib_memo_util(n-2, memo)
            memo[n] = result
            return result
    
    return fib_memo_util(n, memo)

# non-recursive bottom_up approach, reduce space to O(1)
def fib_bottom_up(n):
    if n == 1 or n == 2:
        return 1
    else:
        fib_pre1 = 1
        fib_pre2 = 1
        
        for i in range(3, n+1):
            fib_n = fib_pre1 + fib_pre2
            fib_pre2 = fib_pre1
            fib_pre1 = fib_n
        
        return fib_n

In [2]:
%timeit fib(15)

100 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
%timeit fib_memo(15)

3.44 µs ± 92.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
%timeit fib_bottom_up(15)

594 ns ± 6.55 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
# Climb Stairs
# How many distinct ways can you climb to the top (n), when each time you can either climb 1 or 2 steps?
def climb_stairs(n):
    # base cases:
    if n == 0:
        return 1
    elif n == 1:
        return 1
    
    # bottom up
    else:
        prev1 = 1
        prev2 = 1
        for i in range(2, n + 1):
            curr = prev1 + prev2
            prev2 = prev1
            prev1 = curr
        return curr
    
climb_stairs(4)

5

In [6]:
# House Robber (no robbing of consecutive houses)
def rob(nums) :
    if not nums or len(nums) == 0:
        return 0
    elif len(nums) == 1:
        return nums[0]
    elif len(nums) == 2:
        return max(nums)

    else:
        prev2 = nums[0]
        prev1 = max(nums[0], nums[1])

        for i in range(2, len(nums)):
            curr = max(prev2 + nums[i], prev1)
            prev2 = prev1
            prev1 = curr
        
        return curr
    
rob([1,1,1,2])

# modification: circular houses
# just take max from rob(nums[1:]) and rob(nums[:n-1])

3

In [7]:
# Other questions: 
# cows giving birth: dp[i] = dp[i-1] + dp[i-3]
# derangement: n letter -> N letter box, dp[i] = (i-1)(dp[i-2]+dp[i-1]) when (n>2)

## - 0/1 Knapsack
Select a set of most valuable items within given weight constraints. 
Complete search - O(2^n)

In [51]:
# assume weights are positive integers
def knapsack(max_weight, weights, values):
    # consider 0 max_weight in the dp matrix
    # add in a dummy item with 0 weight& value
    num_items = len(weights)
    dp_mat = [[0 for _ in range(max_weight + 1)] for _ in range(num_items + 1)]
    
    # go through every item, build up the dp table
    for i in range(1, num_items+1):
        weight = weights[i-1]
        value = values[i-1]
        
        # go throughe every max_weight configuration
        for j in range(1, max_weight+1):
            
            # if not able to take item, take whatever available from above row
            if j < weight:
                dp_mat[i][j] = dp_mat[i-1][j]
            
            # taking the item means bearing the weight
            # compare not taking the item with taking the item
            else:
                dp_mat[i][j] = max(dp_mat[i-1][j], dp_mat[i-1][j-weight]+value)
    
    #for row in dp_mat: 
    #    print(row)
        
    return dp_mat[num_items][max_weight]

knapsack(5, [1, 2, 3], [60, 100, 120])

220

In [53]:
# space-optimized version, notice the reverse
def knapsack_opt(max_weight, weights, values):
    dp = [0 for _ in range(max_weight+1)]
    
    for i in range(1, len(values)+1):
        weight = weights[i-1]
        value = values[i-1]
        
        for j in range(max_weight, 0, -1):
            if j >= weight:
                dp[j] = max(dp[j], dp[j-weight]+value)

    return dp[max_weight]

knapsack_opt(5, [1, 2, 3], [60, 100, 120])

220

In [74]:
# find if the array can be partitioned into two subsets with equal sum
def can_partition(nums):
    nums_sum = sum(nums)
    if nums_sum % 2 != 0:
        return False
    else:
        max_weight = nums_sum // 2
        
        # dp[i] - are we able to take some elements from 
        #         the input list that sum to i (T/F)
        dp = [True] + [False] * max_weight
        
        for num in nums:
            
            # go reverse form max_weight to num 
            # or everything will become true if there is a 1 in the input 
            for i in range(max_weight, num-1, -1):
                # if dp[i-nums] already can sum to i-num,
                # then dp[i] is definitely True as well 
                dp[i] = dp[i] | dp[i - num]
        
        return dp[max_weight]

In [73]:
can_partition([1,5,5,11])

True

## - Coin Change

In [3]:
# the fewest number of coins to make up the amount
# dp array: min # coins using 1/2/5 as the last coin
def min_coins(coins, amount):
    # 0 amount has 0 ways, init rest of the array with an invalid num
    dp = [0] + [amount + 1] * amount
    
    for a in range(1, amount + 1):
        for c in coins:
            if c <= a:
                # 1 means taking this coin c, from c amount away
                dp[a] = min(dp[a], dp[a - c] + 1)
    if dp[amount] < (amount + 1):
        return dp[amount]
    else:
        return -1

In [5]:
min_coins([1,2,5], 11)  # 5 + 5 + 1

3

In [6]:
# the total number of ways to make up the amount
def num_ways(coins, amount):
    # dp[0] is 1 because 0 amount can be made up in one way - 0 coins
    dp = [1] + [0] * amount
    
    for c in coins:
        for a in range(c, amount + 1):
            # get from the row above, and add the ways using the new icon
            # similar to knapsack problem
            dp[a] += dp[a-c]
    
    return dp[amount]

In [8]:
num_ways([1,2,5], 5)  # 11111, 1112, 122, 5

4

## - Best time to buy and sell stock
Given an array for which the ith element is the price of a given stock on day i. Find max profit. (must sell the stock before buying again)

In [18]:
# 1. only at most one transaction (buy one and sell one share)
# non-dp way, one-pass solution, maintain two variable
def max_profit_1(prices):
    if len(prices) <= 1:
        return 0
    else:  
        # maintain two values
        max_profit = 0
        lowest_price = prices[0]
        
        # update lowest and max_profit seen so far
        for price in prices[1:]:
            if price < lowest_price:
                lowest_price = price
            elif price - lowest_price > max_profit:
                max_profit = price - lowest_price
        
        return max_profit

In [15]:
max_profit_1([7,1,5,3,6,4])   # 6-1=5

5

In [16]:
# 2. allow multiple transactions (buy and sell multiple times)
# non-dp way, one-pass solution, since multiple transaction allowed,
# keep on adding the profit from every consecutive transaction
def max_profit_2(prices):
    max_profit = 0
    
    for i in range(1, len(prices)):
        if prices[i] > prices[i-1]:
            max_profit += prices[i] - prices[i-1]
    
    return max_profit

In [17]:
max_profit_2([7,1,5,3,6,4])  # (5-1) + (6-3) = 7

7

In [40]:
# 3. at most k transactions
""" 
3 variables to take care of:
  - which day 
  - # transactions completed
  - whether has stock on hand
Depending on "whether has stock on hand", we define two 2d dp array:
  - dp0[i][j]: max profit at the end of ith day, j transactions completed, no stock on hand
  - dp1[i][j]: max profit at the end of ith day, j transactions completed, with stock on hand
dp0 is initialized to 0, dp1 is initialized to -price[i]. 
dp0[n-1][k] is the solution we want, with k transactions allowed and completed.
Initial states: (i == 0 or j == 0)
    when i == 0 and j >= 0: 
      - dp0[0][j] = 0
      - dp1[0][j] = -prices[0]
    when i > 0 and j == 0:
      - dp0[i][0] = 0
      - dp1[i][0] = max(dp[i-1][0], -prices[i])
State transition:
  - dp0[i][j] = max(dp0[i-1][j], dp1[i-1][j-1] + prices[i])  (rest, or sell)
  - dp1[i][j] = max(dp1[i-1][j], dp0[i-1][j] - prices[i])  (rest, or buy in)
(if processing fee is required, substract fee when sell)
"""

def max_profit_3(prices, k):
    # when k is big, same as multiple transactions (no restriction)
    n = len(prices)
    if k >= len(prices)//2:
        return max_profit_2(prices)
    
    dp0 = [[0] * (k+1) for _ in range(n)]
    dp1 = [[0] * (k+1) for _ in range(n)]

    for j in range(k+1):
        dp1[0][j] = -prices[0]

    for i in range(1, n):
        dp1[i][0] = max(dp1[i-1][0], -prices[i])
        
        for j in range(1, k+1):
            dp0[i][j] = max(dp0[i-1][j], dp1[i-1][j-1] + prices[i])
            dp1[i][j] = max(dp1[i-1][j], dp0[i-1][j] - prices[i])
    
    return dp0[n-1][k]

In [41]:
max_profit_3([2,4,1], 2)

2

In [48]:
# 4. cool-down period of 1 day required after selling, k=inf
def max_profit_4(prices):
    
    n = len(prices)
    
    buy = [0] * (n+1)
    buy[1] = -prices[0]
    
    sell = [0] * (n+1)
    
    for i in range(2, n+1):
        buy[i] = max(buy[i-1], sell[i-2] - prices[i-1])
        sell[i] = max(sell[i-1], buy[i-1] + prices[i-1])

    return sell[n]

In [49]:
max_profit_4([1,2,3,0,2])

3